In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
from prompt import time_series_to_image
from utils import view_base64_image, display_messages, collect_results
from data.synthetic import SyntheticDataset

data_name = 'point'
data_dir = f'data/synthetic/{data_name}/eval/'
train_dir = f'data/synthetic/{data_name}/train/'
eval_dataset = SyntheticDataset(data_dir)
eval_dataset.load()
train_dataset = SyntheticDataset(train_dir)
train_dataset.load()

In [ ]:
from utils import plot_series_and_predictions

idx = 0
directory = f'results/synthetic/{data_name}'
series = eval_dataset[idx][1].numpy()
anomaly_locations = eval_dataset[idx][0].numpy()

raw_results = collect_results(directory, raw=True)
raw_results = {k: v[idx] for k, v in raw_results.items()}
results = collect_results(directory)
results = {k: v[idx] for k, v in results.items()}

fig = plot_series_and_predictions(
    series=series, 
    gt_anomaly_intervals=anomaly_locations,
    anomalies=results
)

In [ ]:
from utils import display_messages

display_messages(raw_results['gpt-4o (0shot-vision)']['request']['messages'])

In [ ]:
print(list(raw_results.values())[0]['response'])

In [ ]:
from utils import interval_to_vector
gt = interval_to_vector(anomaly_locations[0])

In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score
from affiliation.generics import convert_vector_to_events
from affiliation.metrics import pr_from_events
from utils import compute_metrics

data = {}

for name, prediction in results.items():
    metrics = compute_metrics(gt, prediction)
    data[name] = metrics

df = pd.DataFrame(data).T
df


In [ ]:
# Keep only the first four columns
df = df.iloc[:, :4].fillna(0)

In [ ]:
{k: v['response'] for k, v in raw_results.items()}

In [ ]:
from utils import highlight_by_ranking
from IPython.core.display import HTML
styled_df = highlight_by_ranking(df.T).T
HTML(styled_df.to_html(escape=False))


In [ ]:
results = collect_results(directory)

In [ ]:
import numpy as np
import pandas as pd
from tqdm import trange
from utils import compute_metrics, interval_to_vector

# Initialize dictionary to store results
metric_names = ['precision', 'recall', 'f1', 'affi precision', 'affi recall', 'affi f1']

results_dict = {
    key: [[] for _ in metric_names] for key in results.keys()
}

# Initialize DataFrame to store results
df_list = []

for i in trange(0, 400):
    series = eval_dataset[idx][1].numpy()
    anomaly_locations = eval_dataset[idx][0].numpy()
    gt = interval_to_vector(anomaly_locations[0])

    for name, prediction in results.items():
        metrics = compute_metrics(gt, prediction[i])
        for idx, metric_name in enumerate(metric_names):
            results_dict[name][idx].append(metrics[metric_name])

# Convert dictionary to DataFrame
df = pd.DataFrame({k: np.mean(v, axis=1) for k, v in results_dict.items()}, 
                  index=['precision', 'recall', 'f1', 'affi precision', 'affi recall', 'affi f1'])

df.T

In [ ]:
styled_df = highlight_by_ranking(df).T
# To visualize the output in Jupyter Notebook
from IPython.core.display import HTML
HTML(styled_df.to_html(escape=False))

In [ ]:
from utils import styled_df_to_latex

latex_table = styled_df_to_latex(styled_df, "Point Anomalies in Sine Wave", label="point_exp")
print(latex_table)